# Desafio Grão Direto IA

O problema do desafio é um problema de Séries Temporais.

## Etapas:

1) Tratamento de Dados
    - Receber os datasets
    - Tratar valores nulos
    - Tratar outlier
2) Analise de Dados
    - Dados importantes
    - Manipular dados
    - Conferir informações dos dados
    - Insights que podemos ter visualizando os dados
3) Feature store
    - Criar features que vão impactar
    - Manipular essas features
    - Criar novas colunas se necessário
    - O maximo de váriaveis
4) Funil de váriaveis
    - Afunilar as váriaveis criadas
    - Identificar quais realmente são importantes
    - Váriaveis que podem representar melhor que outras
    - Dropar colunas que não ajudam(Então atrapalham)
    - Dividir o dataset
5) Aplicação de modelos de Machine Learning
    - Identificar modelos bons para o caso
    - Random Forest
    - Xgboost
    - Fazer testes com os dados manipulados

## Etapas Mercado

1) Remover colunas que não fazem sentido
2) Tratar valores nulos
3) Preencher Datas faltantes
4) Filtrar dataset para a soja
5) Tratar valor do CBOT e Price, diarios e assim unificar datas
6) Preencher o intervalo de datas

# Importando dados de Mercado do Desafio

In [ ]:
import pandas as pd

url = 'C:\workspace\Desafio Grão Direto IA\Data\mercado-desafio.xlsx'

mercado = pd.read_excel(url)

## Tratando dados

In [ ]:
# mercado_dropna = mercado.dropna()
# mercado_dropna.isnull().sum()
mercado['Date'] = mercado['Date'].interpolate(method='linear')
mercado.isnull().sum()

Recebendo a data minima e maxima

In [ ]:
data_max = mercado.max()['Date']
data_max = str(data_max).replace(' 00:00:00','')
data_min = mercado.min()['Date']
data_min = str(data_min).replace(' 00:00:00','')
data_min, data_max

# Identificando outliers

In [ ]:
milho = mercado[mercado['Product'] == 'Milho']
soja = mercado[
    (mercado['Product'] == 'Soja') &
    (mercado['Origin_state'] == 'MG') &
    (mercado['Origin_city'] == 'Araguari') &
    (mercado['Destination_state'] == 'SP') &
    (mercado['Destination_city'] == 'Santos') &
    (mercado['Company'] == 'Polaris')
]
soja = soja.sort_values(by='Date', ascending=True)
soja.plot(x='Date', y='Price', kind='line', title='Preço do Soja', xlabel='Data', ylabel='Preço')

# Fazendo cotação do Dolár

In [ ]:
import yfinance as yf

dolar = yf.Ticker('USDBRL=X')
dolar_hist = dolar.history(interval='1d', start=data_min, end='2024-11-06')
dolar_hist

## Manipulando o dolár

In [ ]:
dolar_hist = pd.DataFrame(dolar_hist,columns=['Close']).reset_index()
dolar_hist['Date'] = dolar_hist['Date'].dt.tz_localize(None)
dolar_hist.info()

Identificando se subiu ou não
- 0 ou False = Não Subiu
- 1 ou True = Subiu

In [ ]:
dolar_hist['Variação'] = dolar_hist['Close'].shift(1) < dolar_hist['Close']
dolar_hist

In [ ]:
mercado = mercado.sort_values(by='Date', ascending=True)
dolar_hist['Variação Percentual'] = ((dolar_hist['Close'] - dolar_hist['Close'].shift(1)) / dolar_hist['Close']) * 100
dolar_hist = dolar_hist.fillna(0)
dolar_hist

# Após arrumar as datas

In [ ]:
mercado_dolar_dados = pd.merge(mercado, dolar_hist, on='Date', how='left')
mercado_dolar_dados
# mdd = mercado[['Date']]
# mdd_unique = mdd.drop_duplicates(subset=['Date'])
# mdd_unique
# datas_diferentes = set(dolar_hist['Date']).difference(set(mdd_unique['Date']))
# datas_diferentes
# 201 rows dolar_hist

## Colunas categoricas:
- Company
- Origin_city
- Origin_state
- Destination_city
- Destination_state
- Product

Utilizando onehotencoding ele produziria muitas colunas. 
- 650 colunas e demorou 6 minutos

In [11]:
from feature_engine import encoding

categoricas = ['Company','Origin_city','Origin_state','Destination_city','Destination_state','Product']

# onehot = encoding.OneHotEncoder(variables=categoricas)
# onehot.fit(mercado_dolar_dados)
# mercado_dolar_dados = onehot.transform(mercado_dolar_dados)
# mercado_dolar_dados

## Soluções
- Podemos utilizar o onehotencoding somente para as colunas mais importantes
- Podemos utilizar o Labelencoder para separar os valores em colunas mais resumidas
- Podemos utilizar o meanencoder para criar colunas com medias dos valores